### Subclassing
In ``Keras`` there are basically three-way we can define a neural network, namely
1. Sequential API
2. Functional API
3. Model Subclassing API
This Notebook will walk through on all these methods in creating one model of the following `achitecture`.

```python
           Input(input_shape=(32, 32, 3)
              |
           Conv2D(64, 3)
              |
           Conv2D(128, 3)
              |
           Conv2D(64, 3)
              |
           MaxPool2D()
              |
           Conv2D (64, 3)
              |
           Conv2D (128, 3)
              |
           Conv2D (64, 3)
              |
           MaxPool2D(2, 2)
              |
           Flatten ()
              |
            Dense (16)
              |
            Dense (output for 10 classes)
```

In [1]:
import tensorflow as tf
import tensorflow.keras as keras

> **Sequential Model**

In [4]:
seq_model = keras.Sequential([
    keras.layers.Input(shape=(32, 32, 3), name="input_layer"),
    keras.layers.Conv2D(64, 3, activation="relu", padding="same", name="conv_1"),
    keras.layers.Conv2D(128, 3, activation="relu", name="conv_2"),
    keras.layers.Conv2D(64, 3, activation="relu", name="conv_3"),
    keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_2d_1"),
    keras.layers.Conv2D(64, 3, activation="relu", name="conv_4"),
    keras.layers.Conv2D(128, 3, activation="relu", name="conv_5"),
    keras.layers.Conv2D(64, 3, activation="relu", name="conv_6"),
    keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_2d_2"),
    keras.layers.Flatten(name="flatten_layer"),
    keras.layers.Dense(64, name="dense_1", activation='relu'),
    keras.layers.Dense(10, name="output_layer", activation='softmax')
], name="seq_model")
seq_model.summary()

Model: "seq_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv_2 (Conv2D)              (None, 30, 30, 128)       73856     
_________________________________________________________________
conv_3 (Conv2D)              (None, 28, 28, 64)        73792     
_________________________________________________________________
pool_2d_1 (MaxPooling2D)     (None, 14, 14, 64)        0         
_________________________________________________________________
conv_4 (Conv2D)              (None, 12, 12, 64)        36928     
_________________________________________________________________
conv_5 (Conv2D)              (None, 10, 10, 128)       73856     
_________________________________________________________________
conv_6 (Conv2D)              (None, 8, 8, 64)          73

> The **Functional Model**

In [7]:
input_layer = keras.layers.Input(shape=(32, 32, 3), name="input_layer")
conv_1 = keras.layers.Conv2D(64, 3, activation="relu", padding="same", name="conv_1")(input_layer)
conv_2 = keras.layers.Conv2D(128, 3, activation="relu", name="conv_2")(conv_1)
conv_3 = keras.layers.Conv2D(64, 3, activation="relu", name="conv_3")(conv_2)
pool_2d_1 =  keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_2d_1")(conv_3)
conv_4 = keras.layers.Conv2D(64, 3, activation="relu", name="conv_4")(pool_2d_1)
conv_5 = keras.layers.Conv2D(128, 3, activation="relu", name="conv_5")(conv_4)
conv_6 = keras.layers.Conv2D(64, 3, activation="relu", name="conv_6")(conv_5)
pool_2d_2 = keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_2d_2")(conv_6)
flatten_layer = keras.layers.Flatten(name="flatten_layer")(pool_2d_2)
dense_1 = keras.layers.Dense(64, name="dense_1", activation='relu')(flatten_layer)
output_layer = keras.layers.Dense(10, name="output_layer", activation='softmax')(dense_1)
fn_model = keras.Model(inputs=input_layer, outputs=output_layer, name="fn_model")
fn_model.summary()

Model: "fn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv_2 (Conv2D)              (None, 30, 30, 128)       73856     
_________________________________________________________________
conv_3 (Conv2D)              (None, 28, 28, 64)        73792     
_________________________________________________________________
pool_2d_1 (MaxPooling2D)     (None, 14, 14, 64)        0         
_________________________________________________________________
conv_4 (Conv2D)              (None, 12, 12, 64)        36928     
_________________________________________________________________
conv_5 (Conv2D)              (None, 10, 10, 128)       738

### The `Model Sub-Classing API`
In `Model Sub-Classing` there are two most important functions **`__init__`** and call. Basically, we will define all the **`tf.keras layers`** or custom implemented layers inside the **`__init__`** method and **`call`** those layers based on our network design inside the call method which is used to perform a forward propagation. The **`call`** method quite the same as the **`forward`** method that is used to build the model in PyTorch anyway.

In [ ]:
> The `Model` class contains the layers in it.

In [48]:
class Model(keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.conv_1 = keras.layers.Conv2D(64, 3, activation="relu", padding="same", name="conv_1")
        self.conv_2 =keras.layers.Conv2D(128, 3, activation="relu", name="conv_2")
        self.conv_3 = keras.layers.Conv2D(64, 3, activation="relu", name="conv_3")
        self.pool = keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_2d_1")
        self.conv_4 = keras.layers.Conv2D(64, 3, activation="relu", name="conv_4")
        self.conv_5 = keras.layers.Conv2D(128, 3, activation="relu", name="conv_5")
        self.conv_6  = keras.layers.Conv2D(64, 3, activation="relu", name="conv_6")
        self.flatten = keras.layers.Flatten(name="flatten_layer")
        self.dense_1 = keras.layers.Dense(64, name="dense_1", activation='relu')
        self.output_layer = keras.layers.Dense(10, name="output_layer", activation='softmax')
        
    def call(self, input_tensor):
        # forward  pass the first block
        x = self.conv_1(input_tensor)
        x = self.conv_2(x)
        x = self.conv_3(x)
        # pool
        x = self.pool(x)
        # forward pass block 2
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = self.conv_6(x)
        # pool block2 
        x = self.pool(x)
        # flatten the conv_6 layer
        x = self.flatten(x)
        # forward pass the output blocks
        x = self.dense_1(x)
        x = self.output_layer(x)
        return x
    
sub_class_model = Model()
sub_class_model.build((None, 32, 32, 3))
sub_class_model.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              multiple                  1792      
_________________________________________________________________
conv_2 (Conv2D)              multiple                  73856     
_________________________________________________________________
conv_3 (Conv2D)              multiple                  73792     
_________________________________________________________________
pool_2d_1 (MaxPooling2D)     multiple                  0         
_________________________________________________________________
conv_4 (Conv2D)              multiple                  36928     
_________________________________________________________________
conv_5 (Conv2D)              multiple                  73856     
_________________________________________________________________
conv_6 (Conv2D)              multiple                  737

> **We still getting the same `Total params` and `Trainable params` just like the from the previous models.** But the problem is we are not able to see the output shape. Which is the problem that we are going to solve later on.

#### Trainning our `model` on the `cifar10` dataset.

In [44]:
from tensorflow.keras import datasets

In [45]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
X_test_tensors = tf.convert_to_tensor(X_test/255., dtype=tf.float32)
X_train_tensors = tf.convert_to_tensor(X_train/255., dtype=tf.float32)
y_test_tensors = tf.one_hot(tf.squeeze(y_test), depth=10)
y_train_tensors = tf.one_hot(tf.squeeze(y_train), depth=10)

In [49]:
seq_model.compile(
    loss = "categorical_crossentropy",
    metrics=['acc'],
    optimizer = 'adam'
)
fn_model.compile(
    loss = "categorical_crossentropy",
    metrics=['acc'],
    optimizer = 'adam'
)
sub_class_model.compile(
    loss = "categorical_crossentropy",
    metrics=['acc'],
    optimizer = 'adam'
)

In [50]:
seq_model.fit(X_train_tensors, y_train_tensors, batch_size=64, epochs=3, verbose=1,
                  validation_data=(X_test_tensors, y_test_tensors)
                  )

Epoch 1/3
782/782 [==============================] - 873s 1s/step - loss: 1.7995 - acc: 0.3323 - val_loss: 1.2974 - val_acc: 0.5139
Epoch 2/3
318/782 [===========>..................] - ETA: 7:39 - loss: 1.2711 - acc: 0.5399

KeyboardInterrupt: 

In [ ]:
fn_model.fit(X_train_tensors, y_train_tensors, batch_size=64, epochs=3, verbose=1,
                  validation_data=(X_test_tensors, y_test_tensors)
                  )

In [ ]:
sub_class_model.fit(X_train_tensors, y_train_tensors, batch_size=64, epochs=3, verbose=1,
                  validation_data=(X_test_tensors, y_test_tensors)
                  )

> The following code was found on the blog post at [towardsdatascience.com](https://towardsdatascience.com/model-sub-classing-and-custom-training-loop-from-scratch-in-tensorflow-2-cc1d4f10fb4e)

In [40]:
class ModelSubClassing(tf.keras.Model):
    def __init__(self, num_classes):
        super(ModelSubClassing, self).__init__()
        # define all layers in init
        # Layer of Block 1
        self.conv1 = tf.keras.layers.Conv2D(32, 3, strides=2, activation="relu")
        self.max1  = tf.keras.layers.MaxPooling2D(3)
        self.bn1   = tf.keras.layers.BatchNormalization()

        # Layer of Block 2
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation="relu")
        self.bn2   = tf.keras.layers.BatchNormalization()
        self.drop  = tf.keras.layers.Dropout(0.3)

        # GAP, followed by Classifier
        self.gap   = tf.keras.layers.GlobalAveragePooling2D()
        self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        # forward pass: block 1 
        x = self.conv1(input_tensor)
        x = self.max1(x)
        x = self.bn1(x)

        # forward pass: block 2 
        x = self.conv2(x)
        x = self.bn2(x)

        # droput followed by gap and classifier
        x = self.drop(x)
        x = self.gap(x)
        return self.dense(x)
model = ModelSubClassing(10)
model.build((None, 32, 32, 3))
model.summary()

Model: "model_sub_classing_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           multiple                  896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0         
_________________________________________________________________
batch_normalization_6 (Batch multiple                  128       
_________________________________________________________________
conv2d_12 (Conv2D)           multiple                  18496     
_________________________________________________________________
batch_normalization_7 (Batch multiple                  256       
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
global_average_pooling2d_3 ( multiple         